In [1]:
import pickle
import pandas as pd
import numpy as np
from IPython.display import Audio
import torch
import torchaudio
from IPython.display import Audio, display
import json
import os
from pathlib import Path

In [2]:
import pickle
import pandas as pd
import numpy as np
from IPython.display import Audio
import torch
import torchaudio
from IPython.display import Audio, display
import json
import os
from pathlib import Path
# import shutil
from glob import glob



def get_df(exp_root, dir):
    exp_dir = Path(exp_root)/dir
    pkl_results_file = os.path.join(exp_dir, "measures.pickle")

    with open(pkl_results_file, "rb") as handle:
        dfme = pd.read_pickle(handle).reset_index(drop=True)
    return dfme


def get_df_storm(pkl_results_file, snr=None):
    # pkl_results_file = "/data/ephraim/datasets/known_noise/sgmse/exp_l/enhanced/storm_vbd/measures_storm.pickle"
    # pkl_results_file = "/data/ephraim/datasets/known_noise/sgmse/exp_k/enhanced/storm_TIMITChime3/measures_storm.pickle"
        # pkl_results_file = "/data/ephraim/datasets/known_noise/sgmse/exp_l/enhanced/sgmse_WSJ0Chime3/measures_storm.pickle"
    # pkl_results_file = "/data/ephraim/datasets/known_noise/sgmse/exp_k/enhanced/sgmse_TIMITChime3/measures_storm.pickle"
    with open(pkl_results_file, "rb") as handle:
        df2 = pd.read_pickle(handle)
    for i in range(len(df2["snr"])):
        df2.at[i, "snr"] = df2["snr"][i].split("_")[0]
    if snr is None:
        df_storm_ = df2.reset_index(drop=True).sort_values(by=['dir'])
    else:
        df_storm_ = df2[df2["snr"]==snr].reset_index(drop=True).sort_values(by=['dir'])
    return df_storm_


def write_results(df, name_alg,analysis_root,noises):
    print("write")
    # df.to_excel(os.path.join(analysis_root, f"{name_alg}_all.xlsx"))
    # for noisetype in noises:
    #     noise_mine  = df[df["noise_type"]==noisetype]
    #     noise_mine.to_excel(os.path.join(analysis_root, f"{name_alg}_{noisetype}.xlsx"))


def drop_rows_without_comparison(df, ours_df):
    for i in range(len(df["dir"])):
        dir_ = df["dir"][i]
        noisetype = df["noise_type"][i]
        mine_parallel = ours_df[(ours_df["dir"]==dir_) & (ours_df["noise_type"]==noisetype)]
        if len(mine_parallel)==0:
            idx = df[(df.dir==dir_)&(df.noise_type==noisetype)].index
            df = df.drop(idx)
        return df


def get_stats_df(df, alg_name, dir_=None, noise_type=None, snr_=None):
    if dir_:
        df = df[df["dir"]==dir_]
    if noise_type:
        df = df[df["noise_type"]==noise_type]
    if snr_:
        df = df[df["snr"]==snr_]
    dfstats = df.describe()[1:3]
    dfstats = dfstats.assign(alg=alg_name)
    return dfstats


def create_mine_df(exp_root,df_noisy,mine,noises,cols,names,snrs):
    for d in names: 
        dfme = get_df(exp_root,d)
        if dfme is None:
            continue
        dfme = dfme.assign(dir=d)
        cur_df_noisy = dfme[dfme["name"]=="noisy"].reset_index(drop=True)
        cur_df_noisy.loc[0,"dir"] = d
        if df_noisy is None:
            df_noisy = cur_df_noisy
        else:
            df_noisy = pd.concat([df_noisy, cur_df_noisy])
        dfme = dfme[dfme["name"] != "noisy"]
        
        dfme = dfme[dfme["stoi"]>0.5]

        for noisetype in noises :
            for c_snr in snrs:
                c_snr=str(c_snr)
                dfme_cur = dfme[dfme["snr"]==c_snr]
                cur_mine = dfme_cur[dfme_cur["noise_type"]==noisetype]
                cur_mine = cur_mine[cur_mine["pesq"] ==cur_mine["pesq"].max()].reset_index(drop=True)[cols]
                if mine is None:
                    mine = cur_mine
                else:
                    mine = pd.concat([mine, cur_mine])
    mine=mine.reset_index(drop=True)
    return mine, df_noisy

# def copy_wavs(df,wavdst,algname):
#     df = df.reset_index(drop=True)
#     for i in df.index.values:
#         wavpath = df["filename"][i]
#         dir_ = df["dir"][i]
#         snr = df["snr"][i]
#         noisetype = df["noise_type"][i]
#         name = f"{dir_}_{noisetype}_snr{snr}_{algname}.wav"
#         dst = wavdst/f"{name}"
#         print("origin:", wavpath)
#         print("dst:", dst)
#         shutil.copyfile(wavpath, dst)


from audio_tools2  import *
def calc_vad(f, verbose=False):
    test_file=f
    fs,s = read_wav(test_file)
    win_len = int(fs*0.025)
    hop_len = int(fs*0.010)
    sframes = enframe(s,win_len,hop_len) # rows: frame index, cols: each frame
    if verbose:
        plot_this(compute_log_nrg(sframes))

    # percent_high_nrg is the VAD context ratio. It helps smooth the
    # output VAD decisions. Higher values are more strict.
    percent_high_nrg = 0.5

    vad = nrg_vad(sframes,percent_high_nrg)

    if verbose:
        plot_these(deframe(vad,win_len,hop_len),s)
    return deframe(vad,win_len,hop_len) 


   



In [15]:
utterance = "6"

In [16]:
net1_path = "/data/ephraim/datasets/known_noise/undiff_exps2/exp_p5_net1/"
def get_net(path_):
    cols =["name","stoi","input_stoi","pesq","input_pesq","OVRL","SIG","BAK","si_sdr",'sdr', "sar",'sir',"noise_type", "filename","snr"]
    df3 = get_df(path_,utterance)[cols]
    df3[df3["pesq"] == df3["pesq"].max()]
    net1 = df3[df3["pesq"] == df3["pesq"].max()][["stoi","pesq","input_stoi","input_pesq"]]
    net1["name"] = path_.split("/")[-2]
    return net1,df3
    

In [17]:
net2_path = "/data/ephraim/datasets/known_noise/undiff_exps2/exp_p5_net3_x5_adapttrain/"
# net3_path = "/data/ephraim/datasets/known_noise/undiff_exps/exp_p4_net3_6c/"
# net4_path = "/data/ephraim/datasets/known_noise/undiff_exps/exp_o3x_net16/"
net1,df3 = get_net(net1_path)
net2,df4 = get_net(net2_path)
# net3,df5 = get_net(net3_path)
# net4,df6 = get_net(net4_path)


In [18]:
df4

,name,stoi,input_stoi,pesq,input_pesq,OVRL,SIG,BAK,si_sdr,sdr,sar,sir,noise_type,filename,snr
0,s0.1,0.819375,0.797242,1.254146,1.109938,2.791389,3.092981,3.815387,8.146991,9.198332,9.198332,inf,6,/data/ephraim/datasets/known_noise/undiff_exps...,5
1,s0.2,0.853966,0.797242,1.276773,1.109938,3.106420,3.411460,3.929449,9.014472,9.681353,9.681353,inf,6,/data/ephraim/datasets/known_noise/undiff_exps...,5
2,s0.23,0.863210,0.797242,1.335136,1.109938,3.182748,3.441780,4.044844,9.094169,9.706152,9.706152,inf,6,/data/ephraim/datasets/known_noise/undiff_exps...,5
3,s0.25,0.862637,0.797242,1.329955,1.109938,3.096262,3.395317,3.925879,9.056033,9.633455,9.633455,inf,6,/data/ephraim/datasets/known_noise/undiff_exps...,5
4,s0.3,0.859853,0.797242,1.332888,1.109938,2.999188,3.399738,3.711205,8.967471,9.458308,9.458308,inf,6,/data/ephraim/datasets/known_noise/undiff_exps...,5
5,s0.35,0.849884,0.797242,1.356504,1.109938,2.935163,3.495058,3.377595,8.870615,9.241170,9.241170,inf,6,/data/ephraim/datasets/known_noise/undiff_exps...,5
6,s0.4,0.849751,0.797242,1.280453,1.109938,2.807667,3.568540,3.144553,8.712801,9.028616,9.028616,inf,6,/data/ephraim/datasets/known_noise/undiff_exps...,5
7,s0.45,0.850268,0.797242,1.301286,1.109938,2.775568,3.494315,3.171593,8.677569,8.980222,8.980222,inf,6,/data/ephraim/datasets/known_noise/undiff_exps...,5
8,s0.5,0.833619,0.797242,1.274993,1.109938,2.752268,3.493784,3.149209,8.428630,8.707900,8.707900,inf,6,/data/ephraim/datasets/known_noise/undiff_exps...,5
9,s0.55,0.843915,0.797242,1.293561,1.109938,2.665526,3.542371,2.907802,8.347813,8.608620,8.608620,inf,6,/data/ephraim/datasets/known_noise/undiff_exps...,5


In [8]:
storm_enhanced_path = os.path.join(net1_path,"storm/enhanced")
storm_results_path = os.path.join(storm_enhanced_path,"sgmse_TIMITChime3/measures_storm.pickle")
sgmse_results_path = os.path.join(storm_enhanced_path,"sgmse_WSJ0Chime3/measures_storm.pickle")
df_storm = get_df_storm(storm_results_path)
df_sg = get_df_storm(sgmse_results_path)


In [9]:
df_sg[df_sg["noise_type"]==utterance][["name","stoi","input_stoi","pesq","input_pesq","OVRL","SIG","BAK","si_sdr",'sdr', "sar",'sir',"noise_type", "filename","snr"]]

,name,stoi,input_stoi,pesq,input_pesq,OVRL,SIG,BAK,si_sdr,sdr,sar,sir,noise_type,filename,snr
0,_noise18_SX137_snr5_file_id_18.wav,0.911095,0.813998,2.038674,1.211043,2.994927,3.343457,3.93577,10.685168,11.142386,11.142386,inf,18,/data/ephraim/datasets/known_noise/undiff_exps...,5


In [10]:
baseline1 = df_sg[df_sg["noise_type"]==utterance][["stoi","pesq","input_stoi","input_pesq"]]
baseline1["name"] = "sgmse_WSJ0Chime3"

In [11]:
df_storm[df_storm["noise_type"]==utterance][["name","stoi","input_stoi","pesq","input_pesq","OVRL","SIG","BAK","si_sdr",'sdr', "sar",'sir',"noise_type", "filename","snr"]]

,name,stoi,input_stoi,pesq,input_pesq,OVRL,SIG,BAK,si_sdr,sdr,sar,sir,noise_type,filename,snr
0,_noise18_SX137_snr5_file_id_18.wav,0.905456,0.813998,1.993211,1.211043,2.916933,3.230453,4.076285,11.228722,11.546936,11.546936,inf,18,/data/ephraim/datasets/known_noise/undiff_exps...,5


In [12]:
baseline2 = df_storm[df_storm["noise_type"]==utterance][["stoi","pesq","input_stoi","input_pesq"]]
baseline2["name"] = "sgmse_TIMITChime3"

In [14]:
print(utterance)
pd.concat([net1,net2,baseline1,baseline2],axis=0)#.to_excel("/data/ephraim/datasets/known_noise/undiff_exps/exp_o2_net18/storm/enhanced/sgmse_TIMITChime3/baseline.xlsx")

18


,stoi,pesq,input_stoi,input_pesq,name
13,0.828130,1.562990,0.813998,1.211043,exp_p5_net1
9,0.830719,1.715758,0.813998,1.211043,exp_p5_net3_x5_adapttrain
0,0.911095,2.038674,0.813998,1.211043,sgmse_WSJ0Chime3
0,0.905456,1.993211,0.813998,1.211043,sgmse_TIMITChime3


In [20]:
print(utterance)
pd.concat([net1,baseline1,baseline2],axis=0)#.to_excel("/data/ephraim/datasets/known_noise/undiff_exps/exp_o2_net18/storm/enhanced/sgmse_TIMITChime3/baseline.xlsx")

18


,stoi,pesq,input_stoi,input_pesq,name
13,0.828130,1.562990,0.813998,1.211043,exp_p5_net1
0,0.923301,1.660742,0.841308,1.092511,sgmse_WSJ0Chime3
0,0.927632,1.751611,0.841308,1.092511,sgmse_TIMITChime3
